In [ ]:
!pip install timm
!pip install fastai==2.5.2

Things added so far: augmentations and cutmix.

Need to add:
1) Metadata!


*needed to update fastai version in order to utilize pytorch datasets/dataloaders without explicit placement of data/model onto Cuda devices: https://forums.fast.ai/t/how-to-use-learner-with-pure-pytorch-dataloader/74990/12?u=ali_baba*

In [ ]:
from fastai.vision.all import *
from torch.utils.data import Dataset, DataLoader
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold

In [ ]:
def get_dls(transforms, batch_size=32, n_folds=5):
    path = Path('../input/petfinder-pawpularity-score')
    train_path = path/'train'
    train_df = pd.read_csv(path/'train.csv')
    train_files, valid_files= get_train_valid_files(path, train_df, 5)
    train_ds = PetDS(train_files, train_df, transforms)
    valid_ds = PetDS(valid_files, train_df, transforms)
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, 
                          num_workers=8, pin_memory=True)
    valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, 
                          num_workers=8, pin_memory=True)
    dls = DataLoaders(train_dl, valid_dl)
    return dls

def get_lbl(sample, df):
    name = sample.name[:-4]
    row = df[df.Id == name]
    lbl = row.Pawpularity.item()
    return lbl

def get_train_valid_files(path, train_df, n_splits):
    skf = KFold(n_splits=5)
    for train_idxs, valid_idxs in skf.split(train_df.Pawpularity):
        break
    files = get_image_files(path/'train')
    train_files = files[train_idxs]
    valid_files = files[valid_idxs]
    return train_files, valid_files
    
class PetDS(Dataset):
    def __init__(self, files, df, transforms):
        self.files = files
        self.df = df
        self.transforms = transforms
    def __len__(self):
        return len(self.files)
    def __getitem__(self, idx):
        sample = self.files[idx]
        lbl = get_lbl(sample, self.df)
        img = np.array(Image.open(sample))
        img = self.transforms(image=img)['image']
        return img.float(), torch.tensor(lbl).float() 

In [ ]:
img_transform = A.Compose([
    A.RandomResizedCrop(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,),
    A.Flip(),
    A.HueSaturationValue(),
    ToTensorV2()
])

In [ ]:
dls = get_dls(img_transform, batch_size=64)

In [ ]:
class Model(nn.Module):
    def __init__(self, n_classes=1, model_name='efficientnet_b0',
                rf=4): #rf -- reduction factor
        super(Model, self).__init__()
        self.m = timm.create_model(model_name, in_chans=3, num_classes=n_classes)
        
        if(model_name[:3] == 'eff'):
            self.feats = self.m.classifier.in_features
            self.m.classifier = nn.Linear(in_features=self.feats, out_features=self.feats//rf)
        else:
            self.feats = self.m.fc.in_features
            self.m.fc = nn.Linear(in_features=self.feats, out_features=self.feats//rf)
        
        self.bn = nn.BatchNorm1d(self.feats//rf)
        self.act = nn.ReLU()
        self.fc = nn.Linear(self.feats//rf, 1)
        
    def forward(self, x):
        out = self.m(x)
        out = self.bn(out)
        out = self.act(out)
        out = self.fc(out)
        return out

In [ ]:
m = Model(model_name='resnet18', rf=2)
learner = Learner(dls, m, loss_func=MSELossFlat(), metrics=rmse,
                 cbs=[MixUp()])
learner.fine_tune(10)